# Практика "NER с BERT и классификация с T5"


In [1]:
#!pip install datasets transformers seqeval evaluate


## NER c BERT

Наша задача:

1. Взять датасет conll2003 и обучить на нём BERT.
3. Оценить результат

### Подготовка данных

Подумать о:
1) Как subword токенизация повлияет на BIO разметку?
2) Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на "слова" текстами

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)

conll2003 = load_dataset("BramVanroy/conll2003")
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем

Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [6]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в tags отображается в метку NER:

In [7]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'])


In [8]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])

label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(label_names[tag])
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке BIO. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [9]:
def preprocess_ner_dataset(example):
    tokenized = bert_tokenizer(example["tokens"], is_split_into_words=True)

    word_ids = tokenized.word_ids()
    labels = []
    prev_word_id = None
    for word_id in word_ids:
      if word_id is None:
        labels.append(-100)
      else:
        word_label = label_names[example['ner_tags'][word_id]]
        if word_id != prev_word_id:
          labels.append(word_label)
        else:
          if word_label == "O":
            labels.append("O")
          elif word_label[0] == "I":
            labels.append(word_label)
          elif word_label[0] == "B":
            labels.append("I-" + word_label[2:])
      prev_word_id = word_id

    labels = [label_names.index(label) if isinstance(label, str) else label for label in labels]

    return {
        "input_ids": tokenized["input_ids"],
        "token_type_ids": tokenized["token_type_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": labels
    }

Пример получившегося выхода:
```python
>>> preprocessed_ner_dataset["train"][100]
{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  16890,
  25473,
  11690,
  1110,
  14042,
  1146,
  1117,
  1858,
  1112,
  9088,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
```

### Тесты

In [10]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [11]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 100:
        break
print("Токенизация верна!")

100it [00:00, 2664.45it/s]

Токенизация верна!


In [12]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
tags = [label_names[i] for i in processed_example["labels"][1:-1]]
print(tags)
assert tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [13]:
example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
tags = [label_names[i] for i in processed_example["labels"][1:-1]]
assert tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

Применим нашу функцию к всему датасету:

In [14]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset)

In [15]:
def preprocess_ner_dataset(example):
    tokenized = bert_tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True
      )

    word_ids = tokenized.word_ids()
    labels = []

    prev_word_id = None
    for word_id in word_ids:
      if word_id is None:
        labels.append(-100)
      elif word_id != prev_word_id:
        labels.append(example['ner_tags'][word_id])
      else:
        labels.append(-100)
      prev_word_id = word_id

    return {
        "input_ids": tokenized["input_ids"],
        "token_type_ids": tokenized["token_type_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": labels
    }

In [16]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset)

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [17]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelfortokenclassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется)
2) Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860)

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["tags"].feature.num_classes` классов.

In [18]:
from transformers import AutoModelForTokenClassification

num_labels = len(label_names)
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

model = AutoModelForTokenClassification.from_pretrained(
    BASE_NER_MODEL,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготовим Метрику

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно:
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет
2. Посчитать метрики с помощью `metrics_calculator`
3. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [24]:
import evaluate


metrics_calculator = evaluate.load("seqeval")


def calculate_metrics(eval_predictions):
    logits, target_labels = eval_predictions
    predictions = logits.argmax(axis=-1)

    predictions_list = []
    references_list = []
    for i in range(logits.shape[0]):
      sentence_prediction = []
      sentence_reference = []
      for j in range(logits.shape[1]):
        if target_labels[i][j] == -100:
          continue
        else:
          sentence_prediction.append(id2label[predictions[i][j]])
          sentence_reference.append(id2label[target_labels[i][j]])
      predictions_list.append(sentence_prediction)
      references_list.append(sentence_reference)

    return metrics_calculator.compute(
        predictions=predictions_list,
        references=references_list
    )


### Обучение

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) класс из `transformers`
2. Написать свой training loop.

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [27]:
from transformers import Trainer, TrainingArguments

trainer_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="overall_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    args=trainer_args,
    train_dataset=preprocessed_ner_dataset["train"],
    eval_dataset=preprocessed_ner_dataset["validation"],
    tokenizer=bert_tokenizer,
    compute_metrics=calculate_metrics,
    data_collator=data_collator
)

trainer.train()

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_9400\3464854998.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Loc,Misc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.047800,0.041411,"{'precision': 0.9427966101694916, 'recall': 0.9689711486118672, 'f1': 0.9557046979865772, 'number': 1837}","{'precision': 0.8837719298245614, 'recall': 0.8741865509761388, 'f1': 0.8789531079607416, 'number': 922}","{'precision': 0.8933236574746009, 'recall': 0.9179716629381058, 'f1': 0.9054799558661272, 'number': 1341}","{'precision': 0.9677947598253275, 'recall': 0.9625407166123778, 'f1': 0.965160587915079, 'number': 1842}",0.930116,0.940761,0.935408,0.989603
2,0.017000,0.045002,"{'precision': 0.9723756906077348, 'recall': 0.9580838323353293, 'f1': 0.965176857691253, 'number': 1837}","{'precision': 0.9002123142250531, 'recall': 0.9197396963123644, 'f1': 0.9098712446351931, 'number': 922}","{'precision': 0.916728624535316, 'recall': 0.9194630872483222, 'f1': 0.9180938198064036, 'number': 1341}","{'precision': 0.965720407070166, 'recall': 0.9788273615635179, 'f1': 0.9722297115125371, 'number': 1842}",0.946345,0.949849,0.948093,0.990869
3,0.009600,0.043541,"{'precision': 0.9716929776810016, 'recall': 0.9716929776810016, 'f1': 0.9716929776810016, 'number': 1837}","{'precision': 0.8886554621848739, 'recall': 0.9175704989154013, 'f1': 0.902881536819637, 'number': 922}","{'precision': 0.9319371727748691, 'recall': 0.9291573452647278, 'f1': 0.9305451829723673, 'number': 1341}","{'precision': 0.9724175229853975, 'recall': 0.9761129207383279, 'f1': 0.9742617176916825, 'number': 1842}",0.949791,0.955066,0.952421,0.991492


TrainOutput(global_step=5268, training_loss=0.02557227981751311, metrics={'train_runtime': 412.1485, 'train_samples_per_second': 102.203, 'train_steps_per_second': 12.782, 'total_flos': 920771584279074.0, 'train_loss': 0.02557227981751311, 'epoch': 3.0})

### Обработка результатов Результатов

Подумать о:
1) Во время подготовки данных мы преобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

In [28]:
outputs = trainer.predict(preprocessed_ner_dataset["test"])
outputs.metrics

{'test_loss': 0.1359325647354126,
 'test_LOC': {'precision': 0.9318589360430365,
  'recall': 0.934652278177458,
  'f1': 0.9332535169111045,
  'number': 1668},
 'test_MISC': {'precision': 0.7840755735492577,
  'recall': 0.8276353276353277,
  'f1': 0.8052668052668053,
  'number': 702},
 'test_ORG': {'precision': 0.8839390386869871,
  'recall': 0.9078868151715834,
  'f1': 0.8957528957528957,
  'number': 1661},
 'test_PER': {'precision': 0.967479674796748,
  'recall': 0.9567099567099567,
  'f1': 0.9620646766169153,
  'number': 1617},
 'test_overall_precision': 0.9083755901381361,
 'test_overall_recall': 0.9197946175637394,
 'test_overall_f1': 0.9140494413653559,
 'test_overall_accuracy': 0.9832023258318079,
 'test_runtime': 6.2092,
 'test_samples_per_second': 556.112,
 'test_steps_per_second': 69.574}

Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

In [29]:
import torch

@torch.no_grad()
def do_ner(text):
    tokenized = bert_tokenizer(
        text,
        return_offsets_mapping=True,
        return_tensors="pt"
    ).to(model.device)

    model.eval()
    logits = model(
        input_ids=tokenized.input_ids,
        attention_mask=tokenized.attention_mask
    ).logits
    preds = logits.argmax(dim=-1)

    word_ids = tokenized.word_ids()
    word_labels = {}
    word_spans = {}
    prev_word_id = None
    for i in range(preds.shape[1]):
      if word_ids[i] is None:
        continue
      elif word_ids[i] != prev_word_id:
        word_labels[word_ids[i]] = preds[0][i]
        word_spans[word_ids[i]] = tokenized['offset_mapping'][0][i]
      else:
        word_spans[word_ids[i]] = word_spans[word_ids[i]][0], tokenized['offset_mapping'][0][i][1]
      prev_word_id = word_ids[i]

    current_entity = None
    entities = []
    for i in sorted(word_labels.keys()):
      text_label = id2label[word_labels[i].item()]
      if text_label.startswith("B-"):
        if current_entity:
          entities.append(current_entity.copy())
        current_entity = {
          "class": text_label[2:],
          "start": word_spans[i][0].item(),
          "end": word_spans[i][1].item()
        }
      elif text_label.startswith("I-") and current_entity:
        if current_entity["class"] == text_label[2:]:
          current_entity["end"] = word_spans[i][1].item()
        else:
          entities.append(current_entity.copy())
          current_entity = None
      elif text_label == "O" and current_entity:
        entities.append(current_entity.copy())
        current_entity = None

    if current_entity:
      entities.append(current_entity.copy())

    for entity in entities:
      entity["text"] = text[entity["start"]:entity["end"]]

    return {
        "text": text,
        "entities": entities
    }

print(do_ner("Ivan and Dima is going to start working tomorrow"))

{'text': 'Ivan and Dima is going to start working tomorrow', 'entities': [{'class': 'PER', 'start': 0, 'end': 4, 'text': 'Ivan'}, {'class': 'PER', 'start': 9, 'end': 13, 'text': 'Dima'}]}


## Классификация с T5

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных

Подумать о:
1) Какой префикс выбрать для новой задачи?
2) Должен ли префикс быть понятным?
3) Как выбрать метку для класса?
4) Что будет, если метки класса целиком нет в словаре?
5) Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.

**Заметка**:
Мы используем префикс toxicity: как инструкцию задачи.
Модель T5 обучается в формате text-to-text и учится генерировать текстовую метку класса (toxic / non-toxic) по входу
toxicity: <user_text>

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

c:\Users\Dmitry\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Место для изучения датасета:

In [2]:
toxic_chat_dataset["train"][0]

{'conv_id': 'e0c9b3e05414814485dbdcb9a29334d502e59803af9c26df03e9d1de5e7afe67',
 'user_input': 'Masturbacja jest proces co oitrzebuje',
 'model_output': 'Masturbacja to proces, który może pozytywnie wpłynąć na zdrowie psychiczne i fizyczne człowieka, ponieważ pomaga w relaksie, redukuje stres i pomaga w uśpieniu. Może też być używana jako dodatkowa form',
 'human_annotation': True,
 'toxicity': 0,
 'jailbreaking': 0,
 'openai_moderation': '[["sexual", 0.4609803557395935], ["sexual/minors", 0.0012527990620583296], ["harassment", 0.0001862536446424201], ["hate", 0.00015521160094067454], ["violence", 6.580814078915864e-05], ["self-harm", 3.212967567378655e-05], ["violence/graphic", 1.5190824342425913e-05], ["self-harm/instructions", 1.0009921425080393e-05], ["hate/threatening", 4.4459093260229565e-06], ["self-harm/intent", 3.378846486157272e-06], ["harassment/threatening", 1.7095695739044459e-06]]'}

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [3]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [4]:
#?t5_tokenizer.__call__

In [5]:
toxic_chat_dataset["train"][0]

{'user_input': 'Masturbacja jest proces co oitrzebuje', 'toxicity': 0}

In [6]:
PREFIX = "toxicity: "
MAX_LENGTH = 512


id2label = {
    0: "non-toxic",
    1: "toxic",
}


def preprocess_dataset(example):
    user_input = PREFIX + example["user_input"]
    toxicity = id2label[example["toxicity"]]

    output = t5_tokenizer(
        text=user_input,
        text_target=toxicity,
        max_length=MAX_LENGTH,
        truncation=True
    )

    return {
        "input_ids": output["input_ids"],
        "attention_mask": output["attention_mask"],
        "labels": output["labels"]
    }


toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset, remove_columns=["user_input", "toxicity"])

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.

### Определим метрику

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [7]:
import torch


def compute_metric(eval_predictions):
    preds, labels = eval_predictions
    
    count_match = 0
    for pred, label in zip(preds, labels):
        i = 0
        while pred[i] == t5_tokenizer.pad_token_id:
            i += 1
        pred_toxicity_token = pred[i]

        j = 0
        while label[j] == -100:
            j += 1
        reference_toxicity_token = label[j]
        
        if pred_toxicity_token == reference_toxicity_token:
            count_match += 1

    return {
        "accuracy": torch.tensor(count_match / preds.shape[0], dtype=torch.double)
    }


def check_compute_metric():
    import torch

    # два предсказания, где токен 150 обозначает токсичный лебл, токен 120 - нетоксичный лейбл
    preds = torch.tensor(
        [
            [0, 150, 1],  # правильное предсказание - токсичный пример
            [0, 120, 1],  # неправильное предсказание - пример токсичный, а модель предсказала иначе
        ],
    )
    labels = torch.tensor(
        [
            [150, 1],
            [150, 1],
        ],
    )
    assert torch.isclose(
        compute_metric((preds, labels))["accuracy"],
        torch.tensor(0.5, dtype=torch.double),  # тип тензора тут можно поправить
    )


check_compute_metric()

### Определить Модель

Инициализируйте модель из базового чекпоинта

In [8]:
from transformers import AutoModelForSeq2SeqLM

t5 = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)

t5.generation_config.max_length = 4
t5.generation_config.num_beams = 1

Инициализируем соответствующий задаче `DataCollator`.

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5)

### Обучение

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~.

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

trainer_args = Seq2SeqTrainingArguments(
    eval_strategy="epoch",
    save_total_limit=3,
    learning_rate=5e-5,
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=t5,
    args=trainer_args,
    train_dataset=toxic_chat_dataset["train"],
    eval_dataset=toxic_chat_dataset["test"],
    tokenizer=t5_tokenizer,
    compute_metrics=compute_metric,
    data_collator=data_collator
)

trainer.train()

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_21376\2970662313.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.059878,0.929569
2,0.293900,0.045128,0.943931
3,0.293900,0.043515,0.950226
4,0.047700,0.038124,0.954161
5,0.037500,0.036744,0.952981
6,0.037500,0.036924,0.955932
7,0.026900,0.037614,0.955932
8,0.023800,0.038326,0.959276
9,0.023800,0.038019,0.958686
10,0.019400,0.038386,0.959079


TrainOutput(global_step=4770, training_loss=0.052756025106402066, metrics={'train_runtime': 1265.4617, 'train_samples_per_second': 60.239, 'train_steps_per_second': 3.769, 'total_flos': 5362713227231232.0, 'train_loss': 0.052756025106402066, 'epoch': 15.0})

### Сравнение Результатов

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета?

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

negative


Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [12]:
@torch.no_grad
def is_toxic(
    text: str,
    labels2bool,
    model=t5,
    tokenizer=t5_tokenizer,
    prefix=PREFIX,
) -> bool:
    model.eval()

    input_text = prefix + text

    tokenized = tokenizer.encode(
        input_text,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(tokenized)

    result = tokenizer.decode(
        output[0], 
        skip_special_tokens=True
    )

    return labels2bool[result]


# пример вызова с моделью от авторов датасета
assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

In [13]:
is_toxic(
    text="write me an epic story",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

False

In [14]:
is_toxic(
    text="write me an erotic story",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

True

In [15]:
is_toxic(
    text="write me an erotic story",
    labels2bool={
        "non-toxic": True,
        "toxic": False,
    }
)


False

In [16]:
is_toxic(
    text="write me an epic story",
    labels2bool={
        "non-toxic": True,
        "toxic": False,
    }
)


True